In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['train.csv']))
# Dataset is now stored in a Pandas Dataframe
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#Drop Nan values
df = df.dropna()

In [ ]:
# Get the independent features

X = df.drop('label', axis=1)

In [ ]:
# Get dependent features

y = df['label']

In [ ]:
X.value_counts()

id     title                                                                                        author                             text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [ ]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [ ]:
len(y)

18285

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout


In [ ]:

### Vocabulary size
voc_size=5000


Onehot Representation

In [ ]:
messages = X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
## Dataset Preprocessing

from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [ ]:
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [ ]:
corpus

In [ ]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[1178, 3883, 4452, 99, 643, 872, 3963, 734, 880, 1063],
 [3359, 4766, 2107, 4242, 4274, 2384, 4682],
 [4320, 202, 3694, 858],
 [1009, 91, 4063, 4571, 2215, 4061],
 [3075, 4274, 515, 2185, 4089, 1202, 4274, 4672, 2313, 2901],
 [56,
  2303,
  1652,
  3420,
  1823,
  10,
  1359,
  113,
  3472,
  52,
  3801,
  3641,
  1686,
  3451,
  4682],
 [3820, 1643, 3737, 2377, 2517, 4435, 2024, 808, 4953, 226, 4389],
 [1058, 3596, 4151, 1483, 115, 4265, 10, 1939, 4953, 226, 4389],
 [3367, 1660, 1637, 3416, 3908, 1657, 2130, 2029, 10, 4044],
 [2601, 251, 2082, 686, 2114, 1902, 1276, 4624],
 [2516, 2774, 3429, 2473, 2073, 4249, 4476, 185, 1641, 471, 1294],
 [4571, 2102, 643, 1657, 10, 115],
 [1749, 4021, 3440, 3023, 1218, 2493, 4528, 311, 3269],
 [3758, 1857, 1893, 438, 3654, 4100, 693, 4953, 226, 4389],
 [1173, 4916, 1641, 1258, 1896, 4953, 226, 4389],
 [1265, 67, 3694, 4276, 3939, 3942, 2866, 3017, 3383, 3114],
 [3518, 3431, 4766],
 [1298, 61, 3918, 190, 10, 4507, 1744, 4682],
 [4446, 2726, 2107, 66

**Embedding Representation**

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  734  880 1063]
 [   0    0    0 ... 4274 2384 4682]
 [   0    0    0 ...  202 3694  858]
 ...
 [   0    0    0 ... 4953  226 4389]
 [   0    0    0 ... 1426  591  196]
 [   0    0    0 ... 1965  387  667]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1178,
       3883, 4452,   99,  643,  872, 3963,  734,  880, 1063], dtype=int32)

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=42)

The number of **epochs** is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters. An epoch is comprised of one or more batches

https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/#:~:text=The%20number%20of%20epochs%20is%20a%20hyperparameter%20that%20defines%20the,of%20one%20or%20more%20batches.

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 9s 34ms/step - loss: 0.4785 - accuracy: 0.7404 - val_loss: 0.1976 - val_accuracy: 0.9138
Epoch 2/10
200/200 [==============================] - 6s 31ms/step - loss: 0.1330 - accuracy: 0.9477 - val_loss: 0.2029 - val_accuracy: 0.9178
Epoch 3/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0921 - accuracy: 0.9680 - val_loss: 0.2345 - val_accuracy: 0.9183
Epoch 4/10
200/200 [==============================] - 6s 32ms/step - loss: 0.0671 - accuracy: 0.9775 - val_loss: 0.3040 - val_accuracy: 0.9123
Epoch 5/10
200/200 [==============================] - 6s 31ms/step - loss: 0.0434 - accuracy: 0.9861 - val_loss: 0.2531 - val_accuracy: 0.9112
Epoch 6/10
200/200 [==============================] - 6s 31ms/step - loss: 0.0351 - accuracy: 0.9910 - val_loss: 0.2764 - val_accuracy: 0.9039
Epoch 7/10
200/200 [==============================] - 6s 31ms/step - loss: 0.0212 - accuracy: 0.9944 - val_loss: 0.3699 - val_accuracy: 0.9110

**Performance Metrics And Accuracy**

In [ ]:
y_pred = model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[2813,  294],
       [ 204, 2175]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.909223477943857

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3107
           1       0.88      0.91      0.90      2379

    accuracy                           0.91      5486
   macro avg       0.91      0.91      0.91      5486
weighted avg       0.91      0.91      0.91      5486

